In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [82]:
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification,
                          AutoModelForCausalLM)

2023-08-15 18:50:50.943040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 18:50:51.513010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [87]:
from src.params import PATH_CODEPARROT
model_name_or_path = PATH_CODEPARROT
n_labels = 150

# print('Loading configuraiton...')
# model_config = GPT2Config.from_pretrained(
#     pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

model_config2 = AutoModelForCausalLM.from_pretrained(model_name_or_path, num_labels=n_labels)

from transformers import AutoModelForSequenceClassification

model2 = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=model_name_or_path, config=model_config2
)

model2.resize_token_embeddings(len(collate_fn.tokenizer))

# # Get model's tokenizer.
# print('Loading tokenizer...')
# tokenizer = GPT2Tokenizer.from_pretrained(
#     pretrained_model_name_or_path=model_name_or_path)
# # default to left padding
# tokenizer.padding_side = "left"
# # Define PAD Token = EOS Token = 50256
# tokenizer.pad_token = tokenizer.eos_token

# # Get the actual model.
# print('Loading model...')
# model = GPT2ForSequenceClassification.from_pretrained(
#     pretrained_model_name_or_path=model_name_or_path, config=model_config)

Some weights of the model checkpoint at data/codeparrot-small were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at data/codeparrot-small and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(32816, 768)

In [14]:
tokenizer.__dict__

{'init_inputs': (),
 'init_kwargs': {'errors': 'replace',
  'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'pad_token': None,
  'add_prefix_space': False,
  'add_bos_token': False,
  'model_max_length': 1024,
  'special_tokens_map_file': None,
  'name_or_path': 'data/codeparrot-small'},
 'name_or_path': 'data/codeparrot-small',
 '_processor_class': None,
 'model_max_length': 1024,
 'padding_side': 'left',
 'truncation_side': 'right',
 'model_input_names': ['input_ids', 'attention_mask'],
 'clean_up_tokenization_spaces': True,
 'deprecation_warnings': {},
 '_in_target_context_manager': False,
 '_bos_token': '<|endoftext|>',
 '_eos_token': '<|endoftext|>',
 '_unk_token': '<|endoftext|>',
 '_sep_token': None,
 '_pad_

In [63]:
a = set()
a.update('as')
a.update('as')
a.update('eqw')

In [64]:
a

{'a', 'e', 'q', 's', 'w'}

In [59]:
from src.codeparrot.dataloader import UITestsDataset
from tqdm import tqdm

dataset = UITestsDataset()

In [73]:
def collect_new_tokens(self : UITestsDataset):
    for item in tqdm(self.items):
        rawtext = self.item2text(item)
        self.encode_whitespaces(rawtext, self.start_extra_id, self.max_len)

collect_new_tokens(dataset)

100%|██████████| 11327/11327 [00:04<00:00, 2296.97it/s]


In [80]:
a = sorted(list(dataset.whitespace_tokens))
a

['<extratoken_0>',
 '<extratoken_10>',
 '<extratoken_11>',
 '<extratoken_12>',
 '<extratoken_13>',
 '<extratoken_14>',
 '<extratoken_15>',
 '<extratoken_16>',
 '<extratoken_17>',
 '<extratoken_18>',
 '<extratoken_19>',
 '<extratoken_1>',
 '<extratoken_20>',
 '<extratoken_21>',
 '<extratoken_22>',
 '<extratoken_23>',
 '<extratoken_24>',
 '<extratoken_25>',
 '<extratoken_26>',
 '<extratoken_27>',
 '<extratoken_28>',
 '<extratoken_29>',
 '<extratoken_2>',
 '<extratoken_30>',
 '<extratoken_31>',
 '<extratoken_32>',
 '<extratoken_33>',
 '<extratoken_34>',
 '<extratoken_35>',
 '<extratoken_36>',
 '<extratoken_37>',
 '<extratoken_38>',
 '<extratoken_39>',
 '<extratoken_3>',
 '<extratoken_40>',
 '<extratoken_41>',
 '<extratoken_42>',
 '<extratoken_43>',
 '<extratoken_44>',
 '<extratoken_45>',
 '<extratoken_46>',
 '<extratoken_47>',
 '<extratoken_48>',
 '<extratoken_49>',
 '<extratoken_4>',
 '<extratoken_50>',
 '<extratoken_51>',
 '<extratoken_52>',
 '<extratoken_53>',
 '<extratoken_54>',
 '<ex

In [75]:
tokenizer.add_tokens(list(dataset.whitespace_tokens), )

99

In [7]:
tokens, rawtext, label = dataset[0]

In [12]:
# def visualize(self, rawtext: str, default_to_notebook: bool = True, mode: str = 'rawtext'):
#     r"""Visualize text from input

#     Problem: Not working with GPT2Tokenizer

#     Arguments:

#             rawtext (:obj: `str`):
#                 Raw text input.

#             default_to_notebook(:obj: `bool`, `optional`, defaults to :obj: `True`):
#                 output to notebook (or html)

#             mode (:obj: `str`, `optional`, defaults to :obj: `rawtext`):
#                 Options: `text`, `rawtext`

#     """
#     if mode == 'text':
#         text = self.encode_whitespaces(rawtext)
#         self.visualizer(text, default_to_notebook=default_to_notebook)
#     elif mode == 'rawtext':
#         self.visualizer(rawtext, default_to_notebook=default_to_notebook)

dataset.visualize(rawtext, mode='text')

In [30]:
ttt = dataset.encode_whitespaces(rawtext[106:115], dataset.start_extra_id, dataset.max_len)
ttt

'<extratoken_2>y:...'

In [78]:
tokenizer.encode(ttt)

[32863, 89, 26, 1396]

In [39]:
# dataset.encode_whitespaces()
dataset.start_extra_id, dataset.max_len

(0, 100)

In [72]:
tokenizer.__dict__

{'init_inputs': (),
 'init_kwargs': {'errors': 'replace',
  'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
  'pad_token': None,
  'add_prefix_space': False,
  'add_bos_token': False,
  'model_max_length': 1024,
  'special_tokens_map_file': None,
  'name_or_path': 'data/codeparrot-small'},
 'name_or_path': 'data/codeparrot-small',
 '_processor_class': None,
 'model_max_length': 1024,
 'padding_side': 'left',
 'truncation_side': 'right',
 'model_input_names': ['input_ids', 'attention_mask'],
 'clean_up_tokenization_spaces': True,
 'deprecation_warnings': {},
 '_in_target_context_manager': False,
 '_bos_token': '<|endoftext|>',
 '_eos_token': '<|endoftext|>',
 '_unk_token': '<|endoftext|>',
 '_sep_token': None,
 '_pad_

In [81]:
dataset.start_extra_id, dataset.max_len

(0, 50)

In [84]:
import numpy as np
np.arange(5, 2, -1)

array([5, 4, 3])

In [85]:
tokenizer.max_len_single_sentence

1024

In [86]:
tokenizer.model_max_length

1024

In [79]:
from src.codeparrot.dataloader import UITestsLoader, ClassificationCollator
from torch.utils.data import DataLoader

dataset = UITestsLoader()
collate_fn = ClassificationCollator(dataset.classes)

dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn)

In [80]:
for sample in dataloader:
    print(sample)
    break

1 2 <class 'list'>
({'input_ids': tensor([[ 3168,    59,  1556,   323,    63,   466,   628,   773,   937,   199,
          3168,    59,  4445,     8,    67,    63,  3808, 31764,  1874,   199,
           199,  5645, 19293, 21815,   298,    35,     2,  4325,   675,    63,
          1391,  1273,    63, 27674,     8, 32770,    88,    26,   399,   890,
            12, 32770,    89,    26,  2263,   199,     9,  1035,  1253,    85,
           890,   469,  5001,    94, 12539,  4124, 30097, 16628, 32770,     6,
            16,   199,    93,   199,   199,  5645, 19293, 21815,   298,    35,
             2,  4325,   675,    63,  3302,    63,  1273,    63, 27674,     8,
            88,    26,  1253,    85,   890,    12,   612,    26, 19172,  1035,
          1253,    85,   890,   469,  5001,  9949, 32770,    88,   199,    93,
           421,   199,   705,    59,    37,   614,  1690,  2189,  4124, 30097,
         16628, 32768, 31647,  2672,  3022,    15,  3808, 31764,    13, 11286,
            13,   

In [52]:
tokens, encodedlabels, rawtexts, labels  = sample

In [57]:
texts = [collate_fn.encode_whitespaces(rawtext) for rawtext in rawtexts]

In [63]:
out1 = collate_fn.tokenizer.encode(
    text=texts[1], 
    truncation=True,
    padding=True,  # for batch 
    max_length=1024,  # 1024
    return_tensors='pt'
)

In [32]:
a, b = list(zip(*inp, ))

In [33]:
a, b

((1, 1, 1), (2, 2, 2))

In [39]:
le.inverse_transform(a)

array(['dasf', 'dasf', 'dasf'], dtype='<U6')

In [89]:
from tokenizers.tools import EncodingVisualizer

vis = EncodingVisualizer(dataset.tokenizer)

In [91]:
rawtext = dataset.item2text(dataset.items[5])

In [93]:
text = dataset.encode_whitespaces(rawtext)

In [94]:
vis(text)

AttributeError: 'list' object has no attribute 'tokens'

In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [36]:
le.fit(['dasf', 'asdfas', 'fsa'])

LabelEncoder()

In [104]:
import torch
torch.from_numpy(le.transform(['dasf', 'asdfas', 'fsa']))

tensor([1, 0, 2])

In [106]:
le.inverse_transform(torch.tensor([2, 1]))

array(['fsa', 'dasf'], dtype='<U6')